In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [3]:
# Emtpy data frame 
df = pd.DataFrame()

# Read all files and create a unique data frame containing all info, for train set
for i in range(1,15+1):
    file = '../input/48fe736d-d85a-4e97-b362-5f045be3c5ec/train/data_0{}.csv'.format(i)
    df_new = pd.read_csv(file)
    df = df.append(df_new,ignore_index=True)

print('Total entries train set: ',len(df))
df.head()

Total entries train set:  5490


,date,T_AMB,P_AMB,CMP_SPEED,CDP,GGDP,HPT_IT,CDT,LPT_IT,EXH_T,RH,WAR,POWER
0,2021-01-01,1.450440,0.843522,0.000000,0.843522,0.843522,NaN,1.450440,NaN,NaN,81.237441,0.000041,NaN
1,2021-01-02,2.761142,0.843856,7870.729713,7.907587,2.448490,949.263690,258.933367,625.677722,387.749872,74.311313,0.000041,13332.692409
2,2021-01-03,9.270325,0.843413,9898.625866,9.407523,2.816769,984.601577,338.014765,655.857137,413.039467,47.897182,0.000041,13026.684965
3,2021-01-04,14.293265,0.844249,9850.791469,9.121784,2.775070,1014.536922,347.129100,681.701087,434.895488,34.400729,0.000041,12773.507042
4,2021-01-05,12.875213,0.843663,9828.508458,9.138088,2.776577,1008.503746,344.360211,677.018748,431.268990,37.537882,0.000041,12768.092781


In [4]:
# Read file for test set
file = '../input/48fe736d-d85a-4e97-b362-5f045be3c5ec/test/test_data_123.csv'
df_test = pd.read_csv(file) 

print('Total entries test set: ', len(df_test))
df_test.head()

Total entries test set:  366


,date,T_AMB,P_AMB,CMP_SPEED,CDP,GGDP,HPT_IT,CDT,LPT_IT,EXH_T,RH,WAR
0,2021-01-01,0.721750,1.001484,6637.409665,5.832015,2.416738,1099.669907,275.264798,825.126859,605.488705,100.000000,0.000040
1,2021-01-02,1.620703,1.001503,6655.303505,5.836681,2.417710,1104.272571,277.170383,828.722891,608.297352,99.855070,0.000043
2,2021-01-03,3.749595,1.001464,6543.075166,5.623416,2.373094,1124.078913,278.159658,849.668850,629.152504,91.417221,0.000045
3,2021-01-04,5.191938,1.001513,6613.171548,5.688147,2.386764,1128.688645,282.134434,851.746149,629.516608,81.896271,0.000045
4,2021-01-05,4.352311,1.001577,5512.789368,4.925938,2.159933,1124.503274,234.921546,847.820191,625.910410,86.960921,0.000045


In [5]:
# Set index as date so we don't have to deal with it
df = df.set_index('date')
df_test = df_test.set_index('date')

In [6]:
# Handling missing values of df and df_test
df.fillna(0, inplace = True)
df_test.fillna(0, inplace = True)

In [7]:
# Features for best fit.
# Display relation to turbine or ambient
var = pd.DataFrame(df.columns)
dicti = {'date': 0, 'T_AMB': 0, 'P_AMB': 0, 'CMP_SPEED': 1, 'CDP': 1, 'GGDP': 1, \
          'HPT_IT': 1, 'CDT': 1, 'LPT_IT': 1, 'EXH_T': 1, 'RH': 0, 'WAR': 0, 'POWER': 1}
# Map values
var['rel'] = var[0].map(dicti)
# Extract only var with rel to turbine
var = var[var['rel'] == 1]
# Omit power of course
var = var[:-1]
high_corr = list(var[0])
# Remove T bc related to P
high_corr.remove('CDT') 

In [8]:
""""All train set"""

# Split feature and target variable
x = df[high_corr]
y = df['POWER']

# Initialize a Linear Reggression model
linreg = LinearRegression()

# Fit (train) linreg to the train set
linreg.fit(x, y) 

LinearRegression()

In [9]:
# Predictions using the test set
df_test = df_test[high_corr]

y_pred = linreg.predict(df_test)
y_pred = pd.DataFrame(y_pred)

y_pred.head()

,0
0,8164.113202
1,8191.732479
2,7880.761424
3,8023.119621
4,8072.597262


In [10]:
# Let's add dates
y_pred['date'] = df_test.index # take dates from original test set
y_pred.columns =  ['POWER', 'date'] # rename cols names
y_pred = y_pred[['date','POWER']] # rearrange cols

y_pred.head()

,date,POWER
0,2021-01-01,8164.113202
1,2021-01-02,8191.732479
2,2021-01-03,7880.761424
3,2021-01-04,8023.119621
4,2021-01-05,8072.597262


In [11]:
# Fix later on dates bc backwards
y_pred.to_csv('./eng_site_all.csv', index = False)

In [12]:
# Coeff
inter = linreg.intercept_
coef = linreg.coef_
